# **Introducción**

Tomando como base el desarrollo presentado en el repositorio: *https://github.com/AlenSmailovic/EdgeDetectionGPGPU*, se realiza una adaptación del mismo para poder realizar la comparación de ejecución del algoritmo de detección de bordes (Edge Detection) en el entorno <strong>Colab</strong>.

El proyecto compara la implementación de detección de bordes realizada en CPU y GPU para ver qué tan rápido, rentable y eficiente es cada ejecución. En el proyecto original se implementa un filtro Sobel: para la implementación de la CPU es un script de Python y para la implementación de la GPU es un kernel con un contenedor de Python.

La comparación se realiza en cinco tipos de imágenes con diferente resolución: 4K (5120X3200), QHD (3840X2160), FHD (1920X1080), HD (1280X720), VGA (640X480).


# **Armado del ambiente**
-Toma las direcciones web de las imagenes con acceso público en internet, la deja disponible al contexto de ejecución del cuaderno Colab.

-Para la ejecución GPU, se debe instalar en el cuaderno en componente Python para CUDA.

In [3]:
#@title ## -Cargar Imagenes a procesar
url_imagen = "https://github.com/AlenSmailovic/EdgeDetectionGPGPU/blob/master/Images/VGA.png?raw=true"
!wget {url_imagen} -O imagen_VGA.png
url_imagen = "https://github.com/AlenSmailovic/EdgeDetectionGPGPU/blob/master/Images/HD.png?raw=true"
!wget {url_imagen} -O imagen_HD.png
url_imagen = "https://github.com/AlenSmailovic/EdgeDetectionGPGPU/blob/master/Images/FHD.png?raw=true"
!wget {url_imagen} -O imagen_FHD.png
url_imagen = "https://github.com/AlenSmailovic/EdgeDetectionGPGPU/blob/master/Images/QHD.png?raw=true"
!wget {url_imagen} -O imagen_QHD.png
url_imagen = "https://github.com/AlenSmailovic/EdgeDetectionGPGPU/blob/master/Images/4K.png?raw=true"
!wget {url_imagen} -O imagen_4K.png

--2021-07-04 20:26:12--  https://github.com/AlenSmailovic/EdgeDetectionGPGPU/blob/master/Images/VGA.png?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/AlenSmailovic/EdgeDetectionGPGPU/raw/master/Images/VGA.png [following]
--2021-07-04 20:26:12--  https://github.com/AlenSmailovic/EdgeDetectionGPGPU/raw/master/Images/VGA.png
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AlenSmailovic/EdgeDetectionGPGPU/master/Images/VGA.png [following]
--2021-07-04 20:26:13--  https://raw.githubusercontent.com/AlenSmailovic/EdgeDetectionGPGPU/master/Images/VGA.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com

In [4]:
#@title ## -Instalación de Paquete *Python* para CUDA (aplica para la ejecución GPU)
#@markdown ---

!pip install pycuda

     |████████████████████████████████| 1.7MB 9.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 12.8MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627583 sha256=94a5490ce2121bfa781f73b69cebd9f90daa88b3f208f41c23261439049b519a
  Stored in directory: /root/.cache/pip/wheels/d5/55/64/fd4dddcc5f1c25eebd90b5291c3769101dc978c70165685512
Successfully built pycuda
  Created wheel for pytools: filename=pytools-2021.2.7-py2.py3-none-any.whl size=60644 sha256=533d4d70e4d8acbcd272b1cca7aaedf384f6d8f571dba7240032a11b4ec38926
  Stored in directory: /root/.cache/pip/wheels/a0/b5/e5/e65d25997fd77729b9aa214645add18688483e48bbcbab6ffc
Successfully built pytools


# **Desarrollo CPU**

In [5]:
import numpy as np
from PIL import Image
import time
import math

def conv_norm(sobelX, sobelY, pxImg, width, height):

    # Create edge image making a copy of initial image
    edge = pxImg

    # Save the computed magnitude in this array
    lenght = np.zeros((width*height))
    index = -1

    # Convolution of Sobel matrix and current image
    for x in range(1, width-1):
      for y in range(1, height-1):
        # Magnitude for X
        pixel_x = (sobelX[0][0] * pxImg[y-1][x-1][0]) + (sobelX[0][1] * pxImg[y][x-1][0]) + (sobelX[0][2] * pxImg[y+1][x-1][0]) + \
                  (sobelX[1][0] * pxImg[y-1][x][0])   + (sobelX[1][1] * pxImg[y][x][0])   + (sobelX[1][2] * pxImg[y+1][x][0])   + \
                  (sobelX[2][0] * pxImg[y-1][x+1][0]) + (sobelX[2][1] * pxImg[y][x+1][0]) + (sobelX[2][2] * pxImg[y+1][x+1][0])
        # Magnitude for Y
        pixel_y = (sobelY[0][0] * pxImg[y-1][x-1][0]) + (sobelY[0][1] * pxImg[y][x-1][0]) + (sobelY[0][2] * pxImg[y+1][x-1][0]) + \
                  (sobelY[1][0] * pxImg[y-1][x][0])   + (sobelY[1][1] * pxImg[y][x][0])   + (sobelY[1][2] * pxImg[y+1][x][0])   + \
                  (sobelY[2][0] * pxImg[y-1][x+1][0]) + (sobelY[2][1] * pxImg[y][x+1][0]) + (sobelY[2][2] * pxImg[y+1][x+1][0])

        index = index + 1
        # Compute the gradient magnitude
        lenght[index] = (( pixel_x * pixel_x + pixel_y * pixel_y ) ** 0.5)

    # Set pixels value in edge image for edge detection
    index = -1
    for i in range(1, width-1):
        for j in range(1, height-1):
            index = index + 1
            # Make sure of each pixel value is in interval [0, 255]
            if (math.isnan(lenght[index]) | (lenght[index] < 0)):
                lenght[index] = 0
            elif (lenght[index] > 255):
                lenght[index] = 255
            # Set the pixel value in edge image
            edge[j][i] = (lenght[index], lenght[index], lenght[index])
       
    return edge

def sobelOperator( pxImg, width, height ):
    # Sobel matrix for edge detection X & Y
    sobelX = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
    sobelY = np.array([[-1, -2, -1],[0, 0, 0],[1, 2, 1]])

    # Apply sobel convolution for edge detecting
    pxImg = conv_norm(sobelX, sobelY, pxImg, width, height)
            
def grayScale ( pxImg, width, height ):
    for x in range(width):
        for y in range(height):

            # get RGB pixelx value from image
            (r, g, b) = pxImg[y][x]

            # Compute the pixels to obtain a gray scale image
            gray = int (r * 0.2126 + g * 0.7152 + b * 0.0722)

            # Set the pixels with new value
            pxImg[y][x] = (gray, gray, gray)

def cpu_edgeDetection( pathImg ):
    # print the path for input image
    print( pathImg )

    timeStart = time.time()

    # Open image from path
    img = Image.open( pathImg )

    # Convert image to RGB matrix
    rgb_img = img.convert('RGB')
    # Save the image matrix in a numpy array
    pxImg = np.array(rgb_img)

    # Get width and height of image
    width, height = rgb_img.size

    timeConvert = time.time()

    # Algorithm for gray scaling an image
    grayScale(pxImg, width, height)

    # Edge detection in current image
    sobelOperator(pxImg, width, height)

    timeProcessing = time.time()

    # Load the image from numpy array
    rgb_img = Image.fromarray(pxImg, mode = "RGB")
    # Save the image at corresponding path
    rgb_img.save( str("CPUEdge") + pathImg )

    timeSave = time.time()

    print( "Image size : ", rgb_img.size )
    print( "Get and convert time : ", timeConvert - timeStart ) 
    print( "Image processing time : ", timeProcessing - timeConvert )
    print( "Saving time : ", timeSave - timeProcessing )
    print( "Total time : ", timeSave - timeStart )
    
    print( "----------------\n" )


# Call cpu_edgeDetection for each type of image
cpu_edgeDetection( "imagen_VGA.png" )
cpu_edgeDetection( "imagen_HD.png" )
cpu_edgeDetection( "imagen_FHD.png" )
cpu_edgeDetection( "imagen_QHD.png" )
cpu_edgeDetection( "imagen_4K.png" )


imagen_VGA.png
Image size :  (640, 480)
Get and convert time :  0.031789302825927734
Image processing time :  11.265188455581665
Saving time :  0.09688687324523926
Total time :  11.393864631652832
----------------

imagen_HD.png
Image size :  (1280, 720)
Get and convert time :  0.02256631851196289
Image processing time :  33.99376082420349
Saving time :  0.13585495948791504
Total time :  34.15218210220337
----------------

imagen_FHD.png
Image size :  (1920, 1080)
Get and convert time :  0.0719447135925293
Image processing time :  76.523930311203
Saving time :  0.6316297054290771
Total time :  77.22750473022461
----------------

imagen_QHD.png
Image size :  (3840, 2160)
Get and convert time :  0.2907216548919678
Image processing time :  320.01117277145386
Saving time :  2.5041329860687256
Total time :  322.80602741241455
----------------

imagen_4K.png
Image size :  (5120, 3200)
Get and convert time :  0.4960496425628662
Image processing time :  611.0316677093506
Saving time :  3.95530

# **Desarrollo GPU**

In [6]:
import sys
try :
  import pycuda.driver as cuda
except ModuleNotFoundError:
  sys.exit("Error al intentar importar el paquete de CUDA, revisar la sección de \"Armado del ambiente\"")
import pycuda.autoinit
from pycuda.compiler import SourceModule
import os
import numpy as numpy
from PIL import Image
import time
import math

BLOCK_SIZE = 1024

# Se reemplaza la siguiente referencia al archivo por la llamada local
cuda_module = SourceModule(""" 
#include <math.h>

#define MIN_RGB_VALUE 0
#define MAX_RGB_VALUE 255

#define     TILE_WIDTH      32
#define     SOBEL_WIDTH     3
#define     W           	(TILE_WIDTH + SOBEL_WIDTH)

__global__ void shared_sobel_filter(const float * pixin, float * pixout, const int width, const int height)
{	
  float cacheImg[W][W];
  
// Index in actual image
  int idx = (threadIdx.x) + blockDim.x * blockIdx.x;
  
  float sobelMatrix[9] = {-1, 0, 1, -2, 0, 2, -1, 0, 1};
  
  // Destination for pixels in shared memory
  int dest = threadIdx.y * blockDim.y + threadIdx.x;
  int destY = dest / W;
  int destX = dest % W;

  // Perform Sobel convolution
  float px_x = 0;
  float px_y = 0;
    
  // Insert pixels from input image into shared memory
  if(idx < width * height)
        cacheImg[destY][destX] = pixin[idx * 3];
    
  __syncthreads();
    
  for(int y = 0; y < SOBEL_WIDTH; ++y) {
    for(int x = 0; x < SOBEL_WIDTH; ++x) {
      // Magnitude for X
      px_x += cacheImg[threadIdx.y][threadIdx.x + y] * sobelMatrix[x + (y * SOBEL_WIDTH)];
      // Magnitude for Y
      px_y += cacheImg[threadIdx.y][threadIdx.x + y] * sobelMatrix[SOBEL_WIDTH - 1 - y + (x * SOBEL_WIDTH)];
    }
  }
  
  // Compute the gradient magnitude
  float px = (float)(sqrt(px_x * px_x + px_y * px_y));

  // Edge cases of MIN or MAX RGB after the Sobel operator is applied
  if (px < MIN_RGB_VALUE)
    px = MIN_RGB_VALUE;
  else if (px > MAX_RGB_VALUE)
    px = MAX_RGB_VALUE;
  
  // Set the pixel value into the edge image (RGB matrix)
  if(idx < width * height) {
    pixout[idx * 3 + 0] = px;
    pixout[idx * 3 + 1] = px;
    pixout[idx * 3 + 2] = px;
  }
  
  __syncthreads();
}

__global__ void sobel_filter(const float * pixin, float * pixout, const int width, const int height)
{	
  int idx = (threadIdx.x) + blockDim.x * blockIdx.x;
  int idy = (threadIdx.y) + blockDim.y * blockIdx.y;
  
// To detect horizontal lines. This is effectively the dy.
  const int sobelX[3][3] = { {-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1} };
// To detect vertical lines. This is effectively the dy.
  const int sobelY[3][3] = { {-1, -2, -1}, {0, 0, 0}, {1, 2, 1} };

  float px_x = 0;
  float px_y = 0;

  if(idx < width * height) {

    for (int j = 0; j < 3; ++j) {
      for (int i = 0; i < 3; ++i) {
        
        // Index in rows
        const int x = i + idx * 3;
        // Index in colomns
        const int y = j + idy * 3;
        
        const int index = x + y;
        // Magnitude for X
        px_x += pixin[index] * sobelX[i][j];
        // Magnitude for Y
        px_y += pixin[index] * sobelY[i][j];
      }
    }
    
    // Compute the gradient magnitude
    float px = (float)(sqrt(px_x * px_x + px_y * px_y));
    
    // Edge cases of MIN or MAX RGB after the Sobel operator is applied
    if (px < MIN_RGB_VALUE)
      px = MIN_RGB_VALUE;
    else if (px > MAX_RGB_VALUE)
      px = MAX_RGB_VALUE;
    
    // Set the pixel value into the edge image (RGB matrix)
    pixout[idx * 3 + 0] = px;
    pixout[idx * 3 + 1] = px;
    pixout[idx * 3 + 2] = px;
  }
}
  
__global__ void gray_scale(const float * pixin, float * pixout, const int width, const int height)
{
  int idx = (threadIdx.x) + blockDim.x * blockIdx.x;
  
  float px = 0;

  if(idx < width * height) {
    
    // Compute pixels to obtain a grayscale image
    px = 0.2126 * pixin[idx * 3 + 0] + 
          0.7152 * pixin[idx * 3 + 1] + 
          0.0722 * pixin[idx * 3 + 2];
    
    // Save pixel into the output image (RGB matrix)
    pixout[idx * 3 + 0] = px;
    pixout[idx * 3 + 1] = px;
    pixout[idx * 3 + 2] = px;
  }
}
""")

# Prepare the kernel functions to be used in Python
#cuda_module = cuda.module_from_file('kernel.cubin')

gray_scale = cuda_module.get_function('gray_scale')
gray_scale.prepare(['P', 'P', 'I', 'I'])

sobel_filter = cuda_module.get_function('sobel_filter')
sobel_filter.prepare(['P', 'P', 'I', 'I'])

shared_sobel_filter = cuda_module.get_function('shared_sobel_filter')
shared_sobel_filter.prepare(['P', 'P', 'I', 'I'])


def gpu_edgeDetection( pathImg ):
	
	# print the path for input image
    print( pathImg )
    
    timeStart = time.time()
    
    # Open image from path
    img = Image.open( pathImg )
    
    # Convert image to RGB matrix
    rgb_img = img.convert('RGB')
    # Save the image matrix in a numpy array	
    pxImg = numpy.array(rgb_img)
    
    # Get width and height of image
    width, height = rgb_img.size
    # Compute them to have the total size
    size = width * height
    
    # Create new numpy arrays exactly the same like input image
    # Store the image result iside them
    gray_px = numpy.empty_like(pxImg)
    sobel_px = numpy.empty_like(pxImg)
    
    timeConvert = time.time()
    
    #----------- Gray Scale ---------
    # Convert the input image into float matrix
    pxImg = pxImg.astype(numpy.float32)
    # Memory allocation inside the GPU for input image
    px_gpu = cuda.mem_alloc(pxImg.nbytes)
    
    # Copy the input image inside the gpu (host -> device)
    cuda.memcpy_htod(px_gpu, pxImg)
    
    # Make sure that the output image is float format
    # Is an empty array
    gray_px = gray_px.astype(numpy.float32)
    # Memory allocation inside the gpu for output image
    gray_px_gpu = cuda.mem_alloc(gray_px.nbytes)
    
    # Copy the empty array inside the gpu (host -> device)
    # The image result will be saved here
    cuda.memcpy_htod(gray_px_gpu, gray_px)
    
    timeAllocMem1 = time.time()
	
  # [INICIO] - Modificación a parámetro por custeo
    grid = (size + BLOCK_SIZE - 1) / BLOCK_SIZE
    tamanio_grid = numpy.int(grid)
  # [FIN] - Modificación a parámetro por custeo

	# Convert RGB image to gray scale image
	# px_gpu      : input image - RGB image
	# gray_px_gpu : output image - gray scale image
	# width       : width of input image 
	# height      : height of input image
	# block        : kernel block size
	# grid         : kernel grid size
    gray_scale(px_gpu, gray_px_gpu,
         numpy.int32(width),
         numpy.int32(height),
         block = (BLOCK_SIZE, 1, 1),
         #grid  = ((size + BLOCK_SIZE - 1) / BLOCK_SIZE, 1, 1))
         grid = (tamanio_grid,1,1))
    
    timeKernelExec1 = time.time()
	
    #----------- Sobel Filter ---------
    # Make sure that the output image is float format
    # Is an empty array
    sobel_px = sobel_px.astype(numpy.float32)
    # Memory allocation inside the gpu for output image
    sobel_px_gpu = cuda.mem_alloc(sobel_px.nbytes)
	
    # Copy the empty array inside the gpu (host -> device)
    # The image result will be saved here
    cuda.memcpy_htod(sobel_px_gpu, sobel_px)
	
    timeAllocMem2 = time.time()
	
  # [INICIO] - Modificación a parámetro por custeo
    grid = (size + BLOCK_SIZE - 1) / BLOCK_SIZE
    tamanio_grid = numpy.int(grid)
  # [FIN] - Modificación a parámetro por custeo

	# Apply sobel filter to gray scale image
	# gray_px_gpu  : input image - gray scale image
	# sobel_px_gpu : output image - edge detetions
	# width        : width of input image 
	# height       : height of input image
	# block        : kernel block size
	# grid         : kernel grid size
    
    print( "Normal Sobel Filter GPU" )
    sobel_filter(gray_px_gpu, sobel_px_gpu,
          numpy.int32(width),
          numpy.int32(height),
          block = (BLOCK_SIZE, 1, 1),
          #grid  = ((size + BLOCK_SIZE - 1)/BLOCK_SIZE, 1))
          grid = (tamanio_grid,1,1))
        
	# Apply sobel filter to gray scale image
	# gray_px_gpu  : input image - gray scale image
	# sobel_px_gpu : output image - edge detetions
	# width        : width of input image 
	# height       : height of input image
	# block        : kernel block size
	# grid         : kernel grid size
    '''
    print "Shared Memory Sobel Filter GPU"
    shared_sobel_filter(gray_px_gpu, sobel_px_gpu,
          numpy.int32(width),
          numpy.int32(height),
          block = (BLOCK_SIZE, 1, 1),
          grid  = ((size + BLOCK_SIZE - 1)/BLOCK_SIZE, 1))
    '''
	
    timeKernelExec2 = time.time()
	
    # Copy the resultet image from device to host
    # The resulted image can be used now in Python    
    cuda.memcpy_dtoh(sobel_px, sobel_px_gpu)
	
	# Convert the image to uint8 format
    sobel_px = numpy.uint8(sobel_px)
    
    timeConvertGPU = time.time()
	
    # Load the image from numpy array
    rgb_img = Image.fromarray(sobel_px, mode = "RGB")
    # Save the image at corresponding path
    rgb_img.save( str("GPUEdge") + pathImg )
	
    timeSave = time.time()

    print( "Image size : ", rgb_img.size )
    print( "Get and convert time : ", timeConvert - timeStart )
    print( "Allocate memory time : ", timeAllocMem1 - timeConvert + timeAllocMem2 - timeKernelExec1 )
    print( "Kernel execution time : ", timeKernelExec1 - timeAllocMem1 + timeKernelExec2 - timeAllocMem2 )
    print( "Get from GPU and convert time : ", timeConvertGPU - timeKernelExec2 )
    print( "Saving time : ", timeSave - timeConvertGPU )
    print( "Total time : ", timeSave - timeStart )
    
    print( "----------------\n" )


# Call gpu_edgeDetection for each type of image
gpu_edgeDetection( "imagen_VGA.png" )
gpu_edgeDetection( "imagen_HD.png" )
gpu_edgeDetection( "imagen_FHD.png" )
gpu_edgeDetection( "imagen_QHD.png" )
gpu_edgeDetection( "imagen_4K.png" )

imagen_VGA.png
Normal Sobel Filter GPU
Image size :  (640, 480)
Get and convert time :  0.012946605682373047
Allocate memory time :  0.0060749053955078125
Kernel execution time :  0.0017769336700439453
Get from GPU and convert time :  0.0019004344940185547
Saving time :  0.10628890991210938
Total time :  0.12898778915405273
----------------

imagen_HD.png
Normal Sobel Filter GPU
Image size :  (1280, 720)
Get and convert time :  0.020215988159179688
Allocate memory time :  0.01577925682067871
Kernel execution time :  0.00023865699768066406
Get from GPU and convert time :  0.004082918167114258
Saving time :  0.1118621826171875
Total time :  0.15217900276184082
----------------

imagen_FHD.png
Normal Sobel Filter GPU
Image size :  (1920, 1080)
Get and convert time :  0.0682382583618164
Allocate memory time :  0.03459429740905762
Kernel execution time :  0.0002923011779785156
Get from GPU and convert time :  0.008605480194091797
Saving time :  0.5869543552398682
Total time :  0.69868469238

---
# **Métricas**


Los resultados obtenidos son:

**Ejecución CPU de cada Imagen**
*   **VGA-Tamaño (640, 380):** 11,39 segundos
*   **HD-Tamaño (1280, 720):** 34,15 segundos
*   **FHD-Tamaño (1920, 1080):** 77,23 segundos
*   **QHD-Tamaño (3840, 2160):** 322,81 segundos
*   **4K-Tamaño (640, 380):** 615,48 segundos

**Total de ejecución CPU: 1061,06 segundos (~17,68 minutos)**

**Ejecución GPU de cada Imagen**
*   **VGA-Tamaño (640, 380):** 0,13 segundos
*   **HD-Tamaño (1280, 720):** 0,15 segundos
*   **FHD-Tamaño (1920, 1080):** 0,7 segundos
*   **QHD-Tamaño (3840, 2160):** 2,65 segundos
*   **4K-Tamaño (640, 380):** 4,44 segundos

**Total de ejecución GPU: 8,07 segundos**


# **Conclusiones**

Los resultados obtenidos muestran de sobremanera el excelente rendimiento y la mayor eficiencia que tiene una ejecución de este tipo de procesamiento en un modo GPU. La posibilidad de paralelización del trabajo muestra que los tiempos obtenidos con muchísimo mejores en un ambiente GPU.

# **Bibliografía**

*   [1] Repositorio cátedra SOA: https://github.com/wvaliente/SOA_HPC
*   [2] Repositorio Edge Detection: https://github.com/AlenSmailovic/EdgeDetectionGPGPU

